# Homework 3 (10 points)
This homework will make use of pandas dataframese to extract and manipulate metadata of seismic station in the Northern California Seismic Network. The learning objective are: data download from URL, dataframes with pandas, basic data manipulation. Use the tutorials shown in class and the pandas resources (https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html)

### Think like a researcher:

We want to download seismic waveforms from a seismic data archive of specific earthquakes. We am not sure what sensors (seismometers) is operating at that time. The list of stations available in the seismic networkhas more than 6000, that's way too many! So we want to filter only the seismic stations that are relevant for the research.

In [2]:
# Address of the website to download data
url = 'http://ncedc.org/ftp/pub/doc/NC.info/NC.channel.summary.day'

In [3]:
# Useful Python modules
import numpy as np
import pandas as pd
import io
import pickle
import requests
from datetime import datetime, timedelta
from math import cos, sin, pi, sqrt
import matplotlib.pyplot as plt

In [4]:
# Import the data from the website into a Python dataframe
s = requests.get(url).content
data = pd.read_csv(io.StringIO(s.decode('utf-8')), header=None, skiprows=2, sep='\s+', usecols=list(range(0, 13)))
data.columns = ['station', 'network', 'channel', 'location', 'rate', 'start_time', 'end_time', 'latitude', 'longitude', 'elevation', 'depth', 'dip', 'azimuth']

In [4]:
# Transform columns start_time and end_time into datetime format
startdate = pd.to_datetime(data['start_time'], format='%Y/%m/%d,%H:%M:%S')
data['start_time'] = startdate
# Avoid 'OutOfBoundsDatetime' error with year 3000
enddate = data['end_time'].str.replace('3000', '2025')
enddate = pd.to_datetime(enddate, format='%Y/%m/%d,%H:%M:%S')
data['end_time'] = enddate

After discussing with my adviser, we decided than only the following channels are relevant for the work we want to do:

In [5]:
channels = ['BHE', 'BHN', 'BHZ', 'BH1', 'BH2', \
            'EHE', 'EHN', 'EHZ', 'EH1', 'EH2', \
            'HHE', 'HHN', 'HHZ', 'HH1', 'HH2', \
            'SHE', 'SHN', 'SHZ', 'SH1', 'SH2']

## Q1 (2 points)

Filter the dataset to keep only the rows with the channels as defined above. 

In [6]:
data = data.loc[data.channel.isin(channels)]
data.head

<bound method NDFrame.head of      station network channel location   rate           start_time  \
0        AAR      NC     EHZ       --  100.0  1984/01/01,00:00:00   
1        AAR      NC     EHZ       --  100.0  1987/05/01,00:00:00   
2        AAR      NC     SHZ       --   20.0  1994/11/28,00:00:00   
3        AAS      NC     EHZ       --  100.0  1984/11/27,18:45:00   
4        AAS      NC     EHZ       --  100.0  1987/05/01,00:00:00   
...      ...     ...     ...      ...    ...                  ...   
6134     WMP      NC     SHE       --   20.0  1995/07/02,12:00:00   
6135     WMP      NC     SHN       --   20.0  1995/07/02,12:00:00   
6136     WMP      NC     SHZ       --   20.0  1995/03/02,19:00:00   
6137     WMP      NC     SHZ       --   20.0  1995/07/02,12:00:00   
6138     WMP      NC     SHZ       10   20.0  1995/07/02,12:00:00   

                 end_time  latitude  longitude  elevation  depth   dip  \
0     1987/05/01,00:00:00  39.27594 -121.02696      911.0    0.0 -9

## Q2  (2 points)
My earthquake catalog starts on 2007/07/01 and ends on 2009/07/01. I am only interested in stations that started recording before 2007/07/01 and ended recording after 2009/07/01. The dataframe <code>data</code> has the start time and end times defined as <code>datetime</code> objects. That means that we can filter the dataframe columns by comparing the datetime objects. To create a datetime object

In [10]:
s1 = datetime(2007,7,1)
s2 = datetime(2009,7,1)
print(type(s1))

<class 'datetime.datetime'>


Filter the dataset to keep only stations that started recording before 2007/07/01 and ended recording after 2009/07/01. 

In [11]:
data = data.loc[(data.start_time < s1) & (data.end_time >s2)]
data.head()

TypeError: '<' not supported between instances of 'str' and 'datetime.datetime'

The cluster of these repeating earthquakes are located at latitude = 40.09N and longitude = -122.87E. Here is a function to compute the distance from the station to the earthquakes, and to add a column distance to the dataset

In [9]:
# the cluster of earthquakes is centered at the following location:
lat0 = 40.09000
lon0 = -122.87000

a = 6378.136 # radius of the Earth in km.
e = 0.006694470 # ellipticity


dx = (pi / 180.0) * a * cos(lat0 * pi / 180.0) / sqrt(1.0 - e * e * sin(lat0 * pi / 180.0) * sin(lat0 * pi / 180.0))
dy = (3.6 * pi / 648.0) * a * (1.0 - e * e) / ((1.0 - e * e * sin(lat0 * pi / 180.0) * sin(lat0 * pi / 180.0)) ** 1.5)
x = dx * (data['longitude'] - lon0)
y = dy * (data['latitude'] - lat0)

# calculate and fill in the dataframe with the new values
data['distance'] = np.sqrt(np.power(x, 2.0) + np.power(y, 2.0))

<ipython-input-9-9373716ffbb5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['distance'] = np.sqrt(np.power(x, 2.0) + np.power(y, 2.0))


## Q3  (3 points)
We want to keep the stations that are located less than 100 km from my repeating earthquakes. For stations farther away, the signal-to-noise ratio would be too low. Filter the dataset to keep only stations that are within 100 km from the earthquakes.

In [10]:
data = data.loc[data.distance < 100]
data.head()

,station,network,channel,location,rate,start_time,end_time,latitude,longitude,elevation,depth,dip,azimuth,distance
1493,GBB,NC,EHZ,--,100.0,2000-12-06 18:38:00,2025-01-01 00:00:00,39.80127,-122.34550,170.0,0.0,-90.0,0.0,55.029997
1528,GCK,NC,EHZ,--,100.0,2000-06-06 21:58:00,2025-01-01 00:00:00,39.54375,-122.43668,251.0,0.0,-90.0,0.0,71.129241
1628,GFC,NC,EHZ,--,100.0,2001-04-03 23:25:00,2020-03-18 22:53:00,39.32655,-122.28886,64.0,0.0,-90.0,0.0,98.346307
1689,GHM,NC,EHZ,--,100.0,1987-05-01 00:00:00,2025-01-01 00:00:00,39.49545,-122.93096,1456.0,0.0,-90.0,0.0,66.387179
1744,GRO,NC,EHZ,--,100.0,1990-12-13 23:30:00,2025-01-01 00:00:00,39.91684,-122.67117,1261.0,0.0,-90.0,0.0,25.657089


Finally, we want to group the result such that the final result looks organized like this:

|station|network|location|latitude|longitude |elevation|depth|distance |channel    |start_time         |end_time           |
|-------|-------|--------|--------|----------|---------|-----|---------|-----------|-------------------|-------------------|
|KBS 	|NC 	|-- 	 |39.91719|-123.59561|1120.0   |0.0  |64.720762|SHZ        |2002-10-17 00:00:00|2011-10-27 21:25:00|
|KCPB 	|NC 	|-- 	 |39.68631|-123.58242|1261.0   |0.0  |75.502041|HHZ,HHN,HHE|2006-10-18 00:08:00|2010-11-01 22:00:00|

We want one row per station, a against one row per channel. Use the pandas function <code>agg</code> to group the channels of a given station together, and srt with the earliest start date and the latest end date. Do not forget to reset the indices!
You can use the following function to group the channels together:

In [ ]:
def f(x):
    """
    Concatenate channels
    """
    result = '%s' % ','.join(x)
    result = list(set(result.split(',')))
    result = '%s' % ','.join(result)
    return result

In [11]:
data.groupby(['station']).agg({'start_time':lambda x: min(x), 'end_time':lambda x: max(x)})
data.reset_index()
data.head()

,station,network,channel,location,rate,start_time,end_time,latitude,longitude,elevation,depth,dip,azimuth,distance
1493,GBB,NC,EHZ,--,100.0,2000-12-06 18:38:00,2025-01-01 00:00:00,39.80127,-122.34550,170.0,0.0,-90.0,0.0,55.029997
1528,GCK,NC,EHZ,--,100.0,2000-06-06 21:58:00,2025-01-01 00:00:00,39.54375,-122.43668,251.0,0.0,-90.0,0.0,71.129241
1628,GFC,NC,EHZ,--,100.0,2001-04-03 23:25:00,2020-03-18 22:53:00,39.32655,-122.28886,64.0,0.0,-90.0,0.0,98.346307
1689,GHM,NC,EHZ,--,100.0,1987-05-01 00:00:00,2025-01-01 00:00:00,39.49545,-122.93096,1456.0,0.0,-90.0,0.0,66.387179
1744,GRO,NC,EHZ,--,100.0,1990-12-13 23:30:00,2025-01-01 00:00:00,39.91684,-122.67117,1261.0,0.0,-90.0,0.0,25.657089


## Q4  (3 points)

How many stations are left in the dataset? Can you plot them using a mapping toolbox or matplotlib? Please add  axis labels, update the fontsize to 14 points, add a title, and a legend, save the file as a PNG.

In [12]:
data.station.count()

26

In [1]:
params = {'legend.fontsize': 14, \
          'xtick.labelsize':14, \
          'ytick.labelsize':14, \
          'font.size':14}
plt.rcParams.update(params)
plt.scatter(data.longitude, data.latitude, label = "Station")
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Station Location')
plt.grid(True)
plt.legend(bbox_to_anchor = (1, 1), loc = 'upper left')
plt.show()
plt.savefig('Homework3_Station_Location')

NameError: name 'plt' is not defined